In [ ]:
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import openpyxl
import pandas as pd
import requests
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
import re
from pykospacing import spacing

excel_file = openpyxl.Workbook()
excel_sheet = excel_file.active

browser = Chrome()

title = []
links = []

for i in range(1,2):
    browser.get("https://www.fmkorea.com/index.php?mid=best2&listStyle=list&page=" + str(i))
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    result = soup.find('div', {'class': 'content_dummy'}).findAll('td', class_='hotdeal_var8')
    for td in result:
        f_link = td.findAll('a', class_='hx')
        for a in f_link:
            conlink = 'https://www.fmkorea.com' + a.get('href')
            links.append(conlink)

    for i in result:
        i = BeautifulSoup(str(i), 'html.parser')

        fm_title = i.find("a", class_="hx").text
        fm_title = spacing(fm_title)
        fm_title = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]','',fm_title)
        title.append(fm_title)

#print(fm_title)


In [ ]:
texts = []
#본문
for i in links:
    try:
        res = requests.get(i)
        res.raise_for_status()
        res.endcoding = None
        html2 = res.text

        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "content_dummy"})
        parags = contentArea.findAll("div", {"class" : "rd_body clear"})

        content = ""

        for parag in parags:
            content += parag.text
        content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
        content = re.sub('[ㅋ]', '', content)
        content = re.sub('[&nbsp;|\n|\t|\r]', '', content)
        content = re.sub('[\xa0]', '', content)
        content = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]', '', content)
        content = re.sub('\복사','', content)
        content = spacing(content)

        texts.append(content)
   
    except HTTPError as e:
        texts.append('')
    except URLError as e:
        texts.append('')
    except AttributeError as e:
        texts.append('')

#print(texts)

In [ ]:
comments = []
#댓글
for i in links:
    try:
        res = requests.get(i)
        res.raise_for_status()
        res.endcoding = None
        html2 = res.text

        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "content_dummy"})
        parags = contentArea.findAll("div", {"class" : "xe_content"})

        content = ""

        for parag in parags:
            content += parag.text
        content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
        content = re.sub('[ㅋ]', '', content)
        content = re.sub('[&nbsp;|\n|\t|\r]', '', content)
        content = re.sub('[\xa0]', '', content)
        content = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]', '', content)
        content = spacing(content)

        comments.append(content)
        
    except HTTPError as e:
        comments.append('')
    except URLError as e:
        comments.append('')
    except AttributeError as e:
        comments.append('')
        
#print(comments)

In [ ]:
fm_dict = {
    '제목' : title,
    '본문' : texts,
    '댓글' : comments
}
df = pd.DataFrame(fm_dict)
df.to_csv('FM_crawling.csv', index = False, encoding='utf-8-sig')
print(df)